In [1]:
import pandas as pd
import numpy as np

## dataframe 정리

In [212]:
# csv파일들 불러오기
sd_list = []
for i in range(51):
    sd_list.append(pd.read_csv(f"song_info/song_data_{i}.csv"))

In [213]:
# 열 이름 재설정
for sd in sd_list:
    sd.columns = ["lyrics", "song_name", "artist", "etc"]

In [214]:
# Dataframe 하나로 합치기
df = pd.concat(sd_list, ignore_index = True)

In [ ]:
# etc 항목 정리
df["album"] = np.nan
df["date"] = np.nan
df["genre"] = np.nan
for i in range(len(df)):
    df["album"][i] = df["etc"][i].split("\r\n")[1]
    df["date"][i] = df["etc"][i].split("\r\n")[3]
    df["genre"][i] = df["etc"][i].split("\r\n")[5]

In [221]:
# etc 열 제거
df = df.drop(columns = ["etc"])

In [229]:
# 가사 없는 곡 제거
no_lyric_idx = df[df["lyrics"].str.strip() == "가사가 없는 곡입니다"].index
df = df.drop(index = no_lyric_idx)

In [233]:
# 인덱스 재설정
df = df.reset_index()

In [236]:
df = df.drop(columns = ["index"])

In [234]:
df

,index,lyrics,song_name,artist,album,date,genre
0,0,I don’t know why but I wanna goIt’s in my bloo...,Blood Stream,YESEO (예서),Blood Stream,2020.08.17,일렉트로니카
1,1,장난스럽게 난너에게 다가갔지만너에게 다가갔지만넌 그게 싫었던 거야장난스럽게 난너에게...,장난,디너프 (DNEIRF),알파카의 하루,2020.08.17,"랩/힙합, 인디음악"
2,2,sometimes i just wanna be away떠나고 파 어디론가로해가 뜨거...,Get Away (Prod. by Minit) (Feat. 김승민),Holynn,Get Away,2020.08.17,R&B/Soul
3,3,I woke up in the morning yeahI’m searching for...,Kid Rock,"팔로알토 (Paloalto), 저드 (jerd), 레디 (Reddy), 수비 (So...",Legacy,2020.08.16,랩/힙합
4,4,꼬불꼬불 꼬여진 길이복잡하고 어지러워도재미도 있을 거야 신날 거야더 힘차게 걸어보자...,SUPADUPA (천천히 해봐),오마이걸,뽀마이걸,2020.08.16,댄스
...,...,...,...,...,...,...,...
2467,2487,여기 만족하기엔 아직 목 말라너네랑 같이 하긴 내가 너무 빨라올해 득점왕은 나야 난...,RUNNING! (Feat. kitsyojii),SUL (에슈엘),Lazy Tape,2020.04.29,"랩/힙합, 인디음악"
2468,2488,지루한 일상 속에 조금씩 지쳐갈 때 한줄기 햇살 창밖에 비쳐오듯 그 조그만 설레임...,설레임,허지영,설레임,2020.04.29,"인디음악, 포크/블루스"
2469,2489,또 몇 시간째 네가 올 것만 같은 느낌에 멍하니 서있네 고요히 가라앉은 이 세계는 ...,아무것도 아닌 게 되는 게,MISA (미사),아무것도 아닌 게 되는 게,2020.04.29,"R&B/Soul, 인디음악"
2470,2490,아무런 말 하지 마우린 서롤 아니까감추려고 하지 마얼어붙어 버린너와 나의 맘을 조립...,믿음,짱유 (JJANGYOU),파도,2020.04.28,랩/힙합


## 전처리

In [7]:
from konlpy.tag import Okt
okt = Okt()

In [8]:
# okt pos tagger로 형태소 분석
df["morphs"] = np.nan
for i in range(len(df)):
    try:
        df["morphs"][i] = okt.pos(df["lyrics"][i])
    except:
        print(i, df["lyrics"][i])
        break

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Public\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
# 불용어 제거
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
for i in range(len(df)):
    for word in df["morphs"][i]:
        if word[1] in ["Punctuation", "Josa", "Suffix", "Number", "PreEomi", "Conjunction", "Determiner", "Eomi", "Unknown"] or word[0] in stopwords:
            idx = df["morphs"][i].index(word)
            del df["morphs"][i][idx]

In [10]:
df["morphs"]

0       [(I, Alpha), (don, Alpha), (t, Alpha), (know, ...
1       [(장난, Noun), (난너, Noun), (다가갔지만, Verb), (너, No...
2       [(sometimes, Alpha), (i, Alpha), (just, Alpha)...
3       [(I, Alpha), (woke, Alpha), (up, Alpha), (in, ...
4       [(꼬불꼬불, Adverb), (꼬여진, Adjective), (길이, Noun),...
                              ...                        
2467    [(여기, Noun), (만족하기엔, Adjective), (아직, Adverb),...
2468    [(지루한, Adjective), (일상, Noun), (속, Noun), (조금씩...
2469    [(또, Noun), (몇, Noun), (시간, Noun), (네, Noun), ...
2470    [(아무런, Adjective), (말, Noun), (하지, Verb), (마, ...
2471    [(영원한, Adjective), (건, Noun), (없다던, Adjective)...
Name: morphs, Length: 2472, dtype: object

In [64]:
# csv 파일 저장
df.to_csv("song_data_pos.csv")

In [2]:
# csv 파일 불러오기
df = pd.read_csv("song_data_pos.csv")

## tf-idf, 코사인 유사도 기반하여 유사한 노래 찾기

In [13]:
# tf-idf를 위한 임시 열 만들기
df["tmp_text"] = np.nan
for i in range(len(df)):
    df["tmp_text"][i] = " ".join([word[0] for word in df["morphs"][i]])

C:\Users\Public\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
# 같은 노래 이름 제거(중복 행 제거)
df = df.drop_duplicates(["song_name"])

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['tmp_text'])
# 가사 데이터에 대해서 tf-idf 수행
print(tfidf_matrix.shape)

(1968, 38444)


In [35]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [71]:
# 노래 제목 - index 찾기
indices = pd.Series(df.index, index=df['song_name']).drop_duplicates()
print(indices.head(40))

song_name
Blood Stream                               0
장난                                         1
Get Away (Prod. by Minit) (Feat. 김승민)      2
Kid Rock                                   3
SUPADUPA (천천히 해봐)                          4
Wait me up (Feat. Delic'amarr & SANHA)     5
Chains                                     6
사랑이 다녀간다                                   7
뒹굴                                         8
너무 사소한 것들에도 네가 (Feat. KURO)                9
He Said                                   10
나 같은 거랑 함께 하느라 고생했어                       11
새하얀 모래 위 발자국 따라 (Feat. SOKO)              12
너 아니면 안돼                                  13
너와 나(You&Me)                              14
EVA GREEN (Feat. Alt)                     15
아를의 밤                                     16
사랑이란                                      17
Crisis                                    18
Green Light                               19
Aqua (feat. 뮤지)                           20
미스타 부                                     21


In [72]:
idx = indices['보라빛 밤 (pporappippam)']
print(idx)

1120


In [75]:
# 'SUPADUPA (천천히 해봐)' 노래와 유사도가 높은 순으로 나열
sim_scores = list(enumerate(cosine_sim[18]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[0:20]
indices = [i[0] for i in sim_scores]
df.drop(columns = ["lyrics", "Unnamed: 0", "morphs", "tmp_text"]).iloc[indices]

,song_name,artist,album,date,genre
18,Crisis,선우정아,비밀의 숲 2 OST Part.1,2020.08.16,국내드라마
699,sEcREt,sEODo,sEcREt,2020.07.19,"R&B/Soul, 인디음악"
324,World is One,"브린 (Bryn), 이영지, 하선호 (Sandy)",World is One,2020.08.03,랩/힙합
1357,Summer!,"수퍼비, UNEDUCATED KID, 트웰브 (twlv), Yuzion",Dingo X Yng & Rich Records (Part 2),2020.06.19,랩/힙합
199,Uh huh (prod. Wiz World),"Tommy Strate, Wiz World",SAFARI,2020.08.08,랩/힙합
756,All the way up,"브린 (Bryn), 이영지, 하선호 (Sandy)",힙합걸Z,2020.07.16,랩/힙합
1481,"Favorite (Feat. punchnello, SOLE)",그리즐리 (Grizzly),Favorite,2020.06.14,"R&B/Soul, 인디음악"
247,W.A.Y (Where Are You),ENOi,W.A.Y (雨 - WhereAreYou),2020.08.06,댄스
372,Room in a pillow (Feat. H:SEAN),Holmsted,Room in a pillow,2020.08.01,R&B/Soul
2394,꽃동네,연희다방,꽃동네,2020.05.02,"발라드, 인디음악"
